<a href="https://colab.research.google.com/github/luciabarg/mentoria_img/blob/master/2_Generaci%C3%B3n_de_fotos_recortadas_y_tabla_An%C3%A1lisis_y_Curaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h4>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación </h4>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</h3>
<h2>Mentoría: Detección de objetos en imágenes</h2>
</center>


En la siguiente notebook, incorporaremos parte de lo realizado en las entregas 1 de los grupos de estudiantes: 

* [Grupo 1](https://github.com/) : Ana Rusconi, Ignacio Grosso y  Mariano Lucero

* [Grupo 2](https://github.com/leorodriguez/mentoria-diplodatos-grupo2):  Diego Gonzalez, Florencia Caballero, Joel Vivas y Leonardo Rodriguez


La siguiente notebook usa como entorno Colab:

In [ ]:
#vinculamos el Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# librerías necesarias 
import os
from IPython.display import Image as Imge
import pandas as pd
import numpy as np
import glob

#seguimos agregando librerías necesarias:
#las que no tengan, vayan instalándolas
import cv2
import numpy as np

import matplotlib.pyplot as plt
import PIL
from PIL import Image
from os import path

from google.colab.patches import cv2_imshow #en Colab!

import seaborn as sns

import re

import json

<a name="exploratoracion_y_curacion"></a>
#### **2 - Exploración y Curación**

#### <a name="generacion_tabla"></a>
#### **2.1 - Generación imágenes recortadas y tabla de features**

Como en la entrega anterior tuvieron la oportunidad de explorar una sola carpeta del dataset, en esta entrega vamos a utilizar una pequeña muestra de las imágenes para que el análisis sea mas representativo del conjunto de datos total. 

Esta notebook es para que vean como se la genera a las imágenes y la tabla que utilizarán en la siguiente notebook '[3-Análisis_y_Curación-Detección_de_Objetos](https://colab.research.google.com/drive/1E6ZUVrYo_n-YS5Hfc5MdIvB9j03ayJXi?usp=sharing)'

La carpeta con los datos, es compartida en el [Drive](https://drive.google.com/drive/folders/1_2oDQPL7n8QI_YEUOecAHyabwA9Gmud0?usp=sharing) y en el desarrollo de esta entrega con las consignas, se utilizará la misma en el entorno de Colab, pero también pueden descargarla desde el repositorio de Github de la mentoría en la carpeta ['muestra'](https://github.com/luciabarg/mentoria_img) 

In [ ]:
#nos posicionamos en la carpeta muestra:
os.chdir('/content/drive/My Drive/Mentoria/muestra')
! ls -d Dataset-*/

Dataset-M/  Dataset-N/	Dataset-T/


In [ ]:
#leemos el archivo de la tabla:
csv_wlabels_df = pd.read_csv('csv_wlabels_muestra_df.csv', index_col=0)
print(csv_wlabels_df.shape)
csv_wlabels_df.head()


(1668, 2061)


,filename,xmin,xmax,ymin,ymax,Time,Size,Type,Hand_Pixel_Width,Hand_Pixel_Height,Area,guantes,color,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,M_G_Mostrando_Amarillos_0_double_box1.jpg,218,461,464,715,M,G,train,243,251,60993,si,Amarillo,0.740974,1.066019,0.132695,0.224892,0.140456,0.129134,0.201447,1.076733,0.461108,0.299800,0.457940,0.201904,0.569677,0.737381,0.480929,0.200076,0.018410,0.674050,0.028826,0.005310,0.128693,0.592353,0.138541,0.141684,0.363294,0.264674,0.519292,...,0.463567,0.091676,0.300955,0.696112,2.409413,0.536980,0.102273,0.157441,0.165093,1.067619,0.751934,0.231443,0.487437,0.446320,0.822407,0.527523,0.217110,0.489194,1.535306,0.521390,0.599480,0.201182,0.391337,0.040206,1.347440,0.448389,0.462778,0.590751,0.877142,0.009986,0.939221,0.859877,0.981910,0.356366,1.748501,0.288037,0.079172,0.266234,0.078312,0.421612
1,M_G_Mostrando_Amarillos_0_double_box2.jpg,617,865,432,598,M,G,train,248,166,41168,si,Amarillo,1.621832,0.208859,0.627041,0.061717,0.052486,0.064176,0.479256,1.068516,0.509059,0.207385,0.218195,0.169190,0.695486,0.237641,0.146711,0.126334,0.029900,0.148363,0.126833,0.181351,0.044585,0.447270,0.134360,0.113650,1.153900,0.653832,0.041024,...,0.816892,0.027489,0.183116,0.626535,1.734605,0.672962,0.030057,0.101685,0.295544,0.392251,0.874054,0.105285,0.110713,0.421417,1.335695,0.500971,1.139208,0.110803,0.850265,0.618181,0.077352,0.376629,0.319234,0.783792,0.427963,0.133063,0.064981,0.017857,0.320706,0.219192,1.199712,0.808466,0.491182,0.356616,0.812572,0.481357,0.071471,0.572322,0.052072,0.213453
2,M_G_Mostrando_Amarillos_105_double_box1.jpg,875,1169,321,450,M,G,train,294,129,37926,si,Amarillo,0.777015,0.350298,0.523675,0.141681,0.008227,0.364888,0.559975,1.060057,0.648008,0.075809,0.419526,0.200864,0.553840,0.182195,0.244877,0.038389,0.046786,0.020999,0.281995,0.016088,0.135179,0.635030,0.276898,0.014077,0.613297,0.622520,0.117482,...,0.250315,0.025067,0.150927,0.604435,1.067641,0.847809,0.220889,0.298962,0.535717,0.566534,0.531686,0.208911,0.254966,0.229268,1.809620,0.362365,0.468568,0.055350,1.388036,0.884211,0.340101,0.305252,0.343156,1.061346,1.043128,0.004126,0.033861,0.020319,0.621639,0.040598,0.916142,0.246840,0.688755,0.101427,0.917090,0.407893,0.343683,0.428996,0.080831,0.005678
3,M_G_Mostrando_Amarillos_105_double_box2.jpg,502,758,145,307,M,G,train,256,162,41472,si,Amarillo,0.206803,0.032262,0.783582,0.104788,0.198923,0.229943,0.102975,1.872409,0.646370,0.369061,0.323460,0.136720,0.515654,0.139382,0.084579,0.205710,0.007865,0.329643,0.279838,0.096330,0.003207,0.161915,0.546153,0.034820,0.178539,0.245108,0.190008,...,0.146642,0.114052,0.005513,0.370992,0.665213,1.402337,0.162487,0.155748,0.241372,0.702011,0.993891,0.174953,0.551115,0.294757,0.660277,0.545881,0.343379,0.348534,0.342936,0.632171,0.079185,0.238604,0.062875,0.798502,1.009929,0.130553,0.003187,0.115578,0.427998,0.110039,0.992854,0.904692,0.112806,0.488674,1.150740,0.275088,0.254404,0.455159,0.012721,0.091329
4,M_G_Mostrando_Amarillos_114_double_box1.jpg,916,1151,285,463,M,G,train,235,178,41830,si,Amarillo,0.683246,0.319097,0.322898,0.489538,0.152056,0.593756,0.177822,1.093265,0.949665,0.154196,0.645907,0.270926,0.383814,0.204949,0.398145,0.102344,0.060811,0.121956,0.028302,0.120451,0.244606,0.387735,0.251303,0.236359,0.880333,0.466272,0.185311,...,0.652505,0.140662,0.354066,0.571745,2.067905,1.038591,0.099552,0.280820,0.516782,0.820352,0.601326,0.042655,0.119472,0.094523,2.042640,0.790532,0.266744,0.056142,0.620942,0.859705,0.439731,0.290626,0.310608,0.721135,0.988298,0.026431,0.317181,0.199194,0.524838,0.139992,1.304742,0.415517,0.652986,0.141079,1.024287,0.117491,0.007151,0.294575,0.217550,0.240535


Para que se hagan una idea de la cantidad de datos que deberíamos analizar, los exploramos y visualizamos con una tabla:

In [ ]:
#Nos ubicamos en la carpeta general
os.chdir('/content/drive/My Drive/Mentoria')

Tenemos 3 conjuntos de datos, Mañana, Tarde y Noche:

In [ ]:
! ls

Dataset-M  Dataset-N  Dataset-T  muestra


In [ ]:
data_sets_path = "/content/drive/My Drive/Mentoria"
os.chdir('{}'.format(data_sets_path))

In [ ]:
#listamos todos las imágenes del dataset:
list_dir = [i for i in glob.glob('./Dataset-*/*/*/*.jpg')]
len(list_dir)

list_jpg_files = []

for file_jpg in list_dir:
  x = re.search('[A-Z]\w+_.+jpg', file_jpg)
  list_jpg_files.append(x.group())

print(len(list_jpg_files))  # len(list_jpg_files) = 8509

8509


In [ ]:
# acá exploramos todos los archivos 'csv'. 
os.chdir('{}'.format(data_sets_path))

def path(day_time, size, typ):
    return ("{path}/Dataset-{time}/{size}"
    "/{time}-{size}-Labels_{typ}.csv").format(path=data_sets_path,
                                              time=day_time, size=size, typ=typ)
def build_df(day_time, size, typ):
    ds = pd.read_csv(path(day_time, size, typ))
    ds['Time'] = day_time
    ds['Size'] = size
    ds['Type'] = typ
    return ds


types = ["train", "test"]
day_time_list = ["M", "T", "N"]
size_list = ["G", "P"]
frames = [build_df(t, s, typ) for t in day_time_list for s in size_list for typ in types]
total_data = pd.concat(frames, ignore_index=True)
total_data["Hand_Pixel_Width"]  = (total_data["xmax"] - total_data["xmin"])
total_data["Hand_Pixel_Height"] = (total_data["ymax"] - total_data["ymin"])
total_data["Area"] = total_data["Hand_Pixel_Width"] * total_data["Hand_Pixel_Height"]
total_data.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,Time,Size,Type,Hand_Pixel_Width,Hand_Pixel_Height,Area
0,M_G_Trabajando_Mano-Verde_7.jpg,1920,1080,hand,877,262,1121,422,M,G,train,244,160,39040
1,M_G_Trabajando_Mano-Verde_7.jpg,1920,1080,hand,1104,121,1266,278,M,G,train,162,157,25434
2,M_G_Trabajando_Verdes_170.jpg,1920,1080,hand,1001,332,1329,613,M,G,train,328,281,92168
3,M_G_Trabajando_Verdes_170.jpg,1920,1080,hand,1233,289,1353,395,M,G,train,120,106,12720
4,M_G_Trabajando_Manos_28.jpg,1920,1080,hand,812,219,944,320,M,G,train,132,101,13332


In [ ]:
list_filename_csv = total_data.filename.unique().tolist()
len(list_filename_csv)  # len(list_filename_csv) = 8509

8509

El asunto es que tienen los valores de todas las imágenes y en nuestro caso, tenemos solo una porción de los mismos, por lo que nos debemos quedar aquellas filas que se correspondan a los imágenes que tenemos:


In [ ]:
#nos posicionamos en la carpeta muestra:
os.chdir('/content/drive/My Drive/Mentoria/muestra')
! ls

all_jpg_muestra  Dataset-N  list_jpg_muestra.txt
Dataset-M	 Dataset-T  muestra_df.csv


Si bien podríamos ir carpeta por carpeta, o generar la tabla anterior con las carpetas dentro de muestra, podemos tomar la información de la tabla anterior, y elegir solo las filas que nos hace falta juntando todas las imágenes en una misma carpeta:

In [ ]:
# acá juntamos todas las fotos de muestra seleccionadas:
os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra')
! ls -d */

classes/  crop/


In [ ]:
list_jpg_muestra = [i for i in glob.glob('*.jpg')]
print(len(list_jpg_muestra))
list_jpg_muestra[0:5]

951


['M_G_Mostrando_Verdes_193.jpg',
 'M_G_Mostrando_Manos_0.jpg',
 'M_G_Mostrando_Manos_186.jpg',
 'M_G_Mostrando_Manos_103.jpg',
 'M_G_Mostrando_Amarillos_144.jpg']

In [ ]:
#otra forma de listar, usando las carpetas
list_dir = [i for i in glob.glob('./Dataset-*/*/*/*.jpg')]
print(len(list_dir))

list_jpg_muestra = []

for file_jpg in list_dir:
  x = re.search('[A-Z]\w+_.+jpg', file_jpg)
  list_jpg_files.append(x.group())

list_jpg_files[0:5]

951


['T_P_Mostrando_Amarillos_100.jpg',
 'T_P_Mostrando_Mano-Amarillo_15.jpg',
 'T_P_Trabajando_Amarillos_101.jpg',
 'T_P_Mostrando_Amarillos_90.jpg',
 'T_P_Mostrando_Manos_159.jpg']

In [ ]:
##guardamos la lista en un archivo para no tener q generarla todas las veces:
#with open('list_jpg_muestra.txt', 'w') as f:
#    f.write(json.dumps(list_data_muestra))

#abrimos el archivo con la lista:
with open('list_jpg_muestra.txt', 'r') as f:
    list_jpg_muestra = json.loads(f.read())

In [ ]:
#acá separamos las filas de los archivos de la lista:
value_list = list_jpg_muestra
muestra_files = total_data.filename.isin(value_list)
muestra_df = total_data[muestra_files].copy()
#eliminamos columnas innecesarias
muestra_df = muestra_df.drop(['width',	'height',	'class'], axis=1)
muestra_df.shape

(1668, 11)

In [ ]:
#lo guardamos en un archivo por única vez, luego de generarlo:
#muestra_df.to_csv('muestra_df.csv') 

In [ ]:
#volvemos a leer el archivo:
muestra_jpg_df = pd.read_csv('muestra_df.csv', index_col=0)
muestra_jpg_df.head()

,filename,xmin,ymin,xmax,ymax,Time,Size,Type,Hand_Pixel_Width,Hand_Pixel_Height,Area
6,M_G_Mostrando_Mano-Amarillo_11.jpg,1063,206,1293,391,M,G,train,230,185,42550
7,M_G_Mostrando_Mano-Amarillo_11.jpg,304,162,503,351,M,G,train,199,189,37611
10,M_G_Trabajando_Amarillos_45.jpg,909,163,1209,313,M,G,train,300,150,45000
11,M_G_Trabajando_Amarillos_45.jpg,1267,196,1419,314,M,G,train,152,118,17936
12,M_G_Trabajando_Manos_0.jpg,625,243,751,339,M,G,train,126,96,12096


In [ ]:
# identificamos cuantos valores de cada 'categoría' tenemos, agregamos las columnas
# 'guantes' y 'color' acorde a las mismas y las guardamos en un diccionario:
tipos = ['Manos', 'Amarillos', 'Verdes', 'Mano-Verde','Mano-Amarillo']
# recordemos que las categorías 'Mano-Verde' y 'Mano-Amarillo' luego hay q 
# determinar cual fila se corresponde a la mano y cuál al guante:

dict_jpg = {}
for tipo in tipos:
    dict_jpg[tipo] = muestra_jpg_df[muestra_jpg_df['filename'].astype(str).str.contains(tipo)].copy()
    if tipo == 'Manos':
        dict_jpg[tipo]['guantes'] = 'no'
        dict_jpg[tipo]['color'] = 'mano'
    elif tipo == 'Amarillos':
        dict_jpg[tipo]['guantes'] = 'si'
        dict_jpg[tipo]['color'] = 'Amarillo'
    elif tipo == 'Verdes':
        dict_jpg[tipo]['guantes'] = 'si'
        dict_jpg[tipo]['color'] = 'Verde'
    elif tipo == 'Mano-Verde':
        dict_jpg[tipo]['guantes'] = 'no'
        dict_jpg[tipo]['color'] = 'Verde'
    elif tipo == 'Mano-Amarillo':
        dict_jpg[tipo]['guantes'] = 'no'
        dict_jpg[tipo]['color'] = 'Amarillo'
        

In [ ]:
#contabilizamos la cantidad de filas a cada categoría:
tipos = ['Manos', 'Amarillos', 'Verdes', 'Mano-Verde','Mano-Amarillo']

data_tot = []
for tipo in tipos:
  print(tipo+': ', dict_jpg[tipo].shape[0])  
  data_tot.append(dict_jpg[tipo].shape[0])


#la suma debería coincidir con la cantidad de filas del dataset:
print('\n Total de datos:', sum(data_tot))


Manos:  489
Amarillos:  428
Verdes:  409
Mano-Verde:  169
Mano-Amarillo:  173

 Total de datos: 1668


Como en el práctico de pre-análisis, creamos una carpeta para guardar las imágenes recortadas de la región de la mano:

In [ ]:
# las carpetas las creamos una sola vez
#! mkdir crop
#! mkdir crop/Manos
#! mkdir crop/Amarillos
#! mkdir crop/Verdes
#! mkdir crop/Mano-Verde
#! mkdir crop/Mano-Amarillo

In [ ]:
tipos = ['Manos', 'Amarillos', 'Verdes', 'Mano-Verde','Mano-Amarillo']

for tipo in tipos:
  path_chosen = 'crop/'+tipo +'/'
  print(path_chosen)

crop/Manos/
crop/Amarillos/
crop/Verdes/
crop/Mano-Verde/
crop/Mano-Amarillo/


Generamos las imágenes recortadas y guardamos su *csv* correspondiente:

In [ ]:
tipos = ['Manos', 'Amarillos', 'Verdes', 'Mano-Verde','Mano-Amarillo']

for tipo in tipos:
  data_img_df = []
  path_chosen = 'crop/'+tipo +'/'
  print(path_chosen)

  for frame, data in dict_jpg[tipo].groupby('filename'):
    num_mano = 1
    if (data.shape[0] == num_mano):
        row = data.iloc[0]
        img_row_file = row.filename
        #print(img_row_file)
        img  = cv2.imread(img_row_file)
        xmin = row.xmin.item()
        xmax = row.xmax.item()
        ymin = row.ymin.item()
        ymax = row.ymax.item()
        
        time = row.Time
        size = row.Size
        sets = row.Type
        witdh  = row.Hand_Pixel_Width.item()
        height = row.Hand_Pixel_Height.item()	
        area = row.Area.item()
        guantes = row.guantes
        color  = row.color


        img_crop_name = img_row_file[:-4]+'_single_box.jpg'
        path = path_chosen +'/'+img_crop_name
        #print(path)
        #print(img_crop_name)
        data_img = [img_crop_name, xmin, xmax, ymin,ymax, time, size,sets,witdh,height,area,guantes,color]
        #print(data_img)
        data_img_df.append(data_img)
        img_crop = img[ymin:ymax, xmin:xmax] 
        #cv2_imshow(img_crop)
        #cv2.imwrite(path,img_crop)
    else:
        row1 = data.iloc[0]
        img_row1_file = row1.filename
        #print(img_row1_file)
        img_row1=cv2.imread(img_row1_file)
        #cv2_imshow(img_row1)
        xmin_r1 = row1.xmin.item()
        xmax_r1 = row1.xmax.item()
        ymin_r1 = row1.ymin.item()
        ymax_r1 = row1.ymax.item()

        time_r1 = row1.Time
        size_r1 = row1.Size
        sets_r1 = row1.Type
        witdh_r1  = row1.Hand_Pixel_Width.item()
        height_r1 = row1.Hand_Pixel_Height.item()	
        area_r1 = row1.Area.item()
        guantes_r1 = row1.guantes
        color_r1  = row1.color

        img_crop_name_row1 = img_row1_file[:-4]+'_double_box1.jpg'
        img_crop_row1 = img_row1[ymin_r1:ymax_r1, xmin_r1:xmax_r1] 
        path1 = path_chosen+'/'+img_crop_name_row1
        #print(path1)
        data_img1 = [img_crop_name_row1, xmin_r1, xmax_r1, ymin_r1, ymax_r1, 
                    time_r1, size_r1, sets_r1, witdh_r1, height_r1, 
                    area_r1, guantes_r1, color_r1]
        #print(data_img1)
        data_img_df.append(data_img1)
        #cv2.imwrite(path1, img_crop_row1)

        row2 = data.iloc[1]
        img_row2_file = row2.filename
        img_row2=cv2.imread(img_row2_file)
        xmin_r2 = row2.xmin.item()
        xmax_r2 = row2.xmax.item()
        ymin_r2 = row2.ymin.item()
        ymax_r2 = row2.ymax.item()

        time_r2 = row2.Time
        size_r2 = row2.Size
        sets_r2 = row2.Type
        witdh_r2  = row2.Hand_Pixel_Width.item()
        height_r2 = row2.Hand_Pixel_Height.item()	
        area_r2 = row2.Area.item()
        guantes_r2 = row2.guantes
        color_r2  = row2.color


        img_crop_name_row2 = img_row2_file[:-4]+'_double_box2.jpg'
        data_img2 = [img_crop_name_row2, xmin_r2, xmax_r2, ymin_r2,ymax_r2, 
                    time_r2, size_r2,sets_r2,witdh_r2,height_r2,
                    area_r2, guantes_r2, color_r2]
        data_img_df.append(data_img2)

        img_crop_row2 = img_row2[ymin_r2:ymax_r2, xmin_r2:xmax_r2] 
        path2 = path_chosen +'/'+img_crop_name_row2
        #cv2_imshow(img_crop)
        #cv2.imwrite(path2, img_crop_row2)

  cropped_name_df = 'crop/'+tipo +'/'+tipo + '_cropped_df.csv'
  print(cropped_name_df)
  cropped_df = pd.DataFrame(data_img_df,
                     columns = ['filename','xmin', 'xmax','ymin','ymax',
                                'Time', 'Size', 'Type', 'Hand_Pixel_Width', 'Hand_Pixel_Height',
                                'Area', 'guantes', 'color' 
                               ])  
  print(cropped_df.shape[0])
  cropped_df.to_csv(cropped_name_df) 


crop/Manos/
crop/Manos/Manos_cropped_df.csv
489
crop/Amarillos/
crop/Amarillos/Amarillos_cropped_df.csv
428
crop/Verdes/
crop/Verdes/Verdes_cropped_df.csv
409
crop/Mano-Verde/
crop/Mano-Verde/Mano-Verde_cropped_df.csv
169
crop/Mano-Amarillo/
crop/Mano-Amarillo/Mano-Amarillo_cropped_df.csv
173


In [ ]:
#volvemos a leer los archivos:
tipos = ['Manos', 'Amarillos', 'Verdes', 'Mano-Verde','Mano-Amarillo']

for tipo in tipos:
  print(tipo)
  path_chosen = 'crop/'+tipo +'/'
  print(path_chosen)

  cropped_name_df = 'crop/'+tipo +'/'+tipo + '_cropped_df.csv'
  cropped_df = pd.read_csv(cropped_name_df, index_col=0)
  list_filename_df = cropped_df.filename.unique().tolist()
  print('cantidad de fotos: ', len(list_filename_df)) 
  print(cropped_name_df,'shape:', cropped_df.shape, '\n')

Manos
crop/Manos/
cantidad de fotos:  489
crop/Manos/Manos_cropped_df.csv shape: (489, 13) 

Amarillos
crop/Amarillos/
cantidad de fotos:  428
crop/Amarillos/Amarillos_cropped_df.csv shape: (428, 13) 

Verdes
crop/Verdes/
cantidad de fotos:  409
crop/Verdes/Verdes_cropped_df.csv shape: (409, 13) 

Mano-Verde
crop/Mano-Verde/
cantidad de fotos:  169
crop/Mano-Verde/Mano-Verde_cropped_df.csv shape: (169, 13) 

Mano-Amarillo
crop/Mano-Amarillo/
cantidad de fotos:  173
crop/Mano-Amarillo/Mano-Amarillo_cropped_df.csv shape: (173, 13) 



##### **Clases**

Si bien las categorías nos sirven para ser mas ordenada la exploración de imágenes, como ya hemos visto, en realidad tenemos tres *clases*:
 'Mano', 'Amarillo' y 'Verde'

In [ ]:
#! mkdir classes
# para copiar toda una carpeta
#! rsync -r crop/ classes/
#

In [ ]:
# nos movemos a la carpeta de clases:
os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra/classes/')
!ls -d */

Amarillo/  Mano/  Mano-Amarillo/  Mano-Verde/  Verde/


In [ ]:
#acá fui creando y copiando las nuevas carpetas, lo podría haber hecho a mano desde el Drive: 
#! mkdir Mano
#! rsync -r Manos/ Mano/
#! rm -rf Amarillos/
#! rm -rf Verdes/
#! rm -rf Manos/

In [ ]:
# nos movemos a la carpeta de clases:
os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra/classes/')
! ls -d */
! ls -d */*/


Amarillo/  Mano/  Mano-Amarillo/  Mano-Verde/  Verde/
Mano-Amarillo/Amarillo/  Mano-Verde/Mano/
Mano-Amarillo/Mano/	 Mano-Verde/Verde/


Verificamos los archivos y la cantidad de fotos en cada carpeta:

In [ ]:
#volvemos a leer los archivos:
tipos = ['Amarillos', 'Manos', 'Verdes']

for tipo in tipos:
  print(tipo)
  cropped_name_df = tipo + '_cropped_df.csv'
  cropped_df = pd.read_csv(cropped_name_df, index_col=0)
  list_filename_df = cropped_df.filename.unique().tolist()

  print('cantidad de fotos: ', len(list_filename_df)) 
  print(cropped_name_df,'shape:', cropped_df.shape, '\n')

Amarillos
cantidad de fotos:  428
Amarillos_cropped_df.csv shape: (428, 13) 

Manos
cantidad de fotos:  489
Manos_cropped_df.csv shape: (489, 13) 

Verdes
cantidad de fotos:  409
Verdes_cropped_df.csv shape: (409, 13) 



In [ ]:
#Para agrupar los archivos de una carpeta según su nombre, 
#os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra/classes/Amarillo')            # 428 fotos 
#os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra/classes/Mano')                # 489 fotos 
#os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra/classes/Verde')                # 409 fotos 
#extension = 'jpg'
#list_cropped_class = [i for i in glob.glob('*.{}'.format(extension))]
#len(list_cropped_class)

Para las carpetas 'Mano-Verde' y 'Mano-Amarillo':

In [ ]:
# Que lindx que te ves cortando las imágenes en la sección anterior, pero te falta separar de las categorías 'Mano-Verde' y 'Mano-Amarillo'...
# Esto es a ojímetro, si se les ocurre otra forma, bienvenido sea, fíjense todo lo que avanzamos con su aporte :)

In [ ]:
# nos movemos a la carpeta de Mano-Amarillo / Mano-Verde respectivamente:
# os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra/crop/Mano-Amarillo/')            # 173 fotos 
# os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra/classes/Mano-Amarillo/Mano')     # 87 fotos
# os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra/classes/Mano-Amarillo/Amarillo') # 86 fotos

# os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra/crop/Mano-Verde/')               # 169 fotos
# os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra/classes/Mano-Verde/Mano')        # 86 fotos
# os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra/classes/Mano-Verde/Verde')       # 86 fotos


In [ ]:
#Para agrupar los archivos de una carpeta según su nombre 
extension = 'jpg'
list_cropped_class = [i for i in glob.glob('*.{}'.format(extension))]
list_cropped_class.sort()
len(list_cropped_class)

In [ ]:
#esto vamos cambiando respectivamente por amarillos o verdes
#list_mano_amarillo_mano = list_cropped_class.copy()
#len(list_mano_amarillo_mano)
list_mano_verde_mano = list_cropped_class.copy()
len(list_mano_verde_mano)

86

In [ ]:
##guardamos la lista en un archivo para no tener q generarla todas las veces:

#volvemos a la carpeta mano-guante
#os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra/classes/Mano-Amarillo')  # Mano-Amarillo_cropped_df.csv
#list_data_muestra = list_mano_amarillo_mano
#filename = 'list_mano_amarillo_mano.txt'
#print(len(list_data_muestra))

#os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra/classes/Mano-Verde')     # Mano-Verde_cropped_df.csv
#list_data_muestra = list_mano_verde_mano
#filename = 'list_mano_verde_mano.txt'

#with open(filename, 'w') as f:
#    f.write(json.dumps(list_data_muestra))

#abrimos el archivo con la lista:
#with open(filename, 'r') as f:
#    list_jpg_muestra = json.loads(f.read())

Cuando separamos las que son guantes de las manos, se supone que las que no están en la lista, son manos. De todas formas, lo contabilizamos como para verificarlo.

In [ ]:
# A filename_chosen lo vamos cammbiando, dependiendo del csv. 
#filename_mano_guante_ch = 'Mano-Amarillo_cropped_df.csv' 
filename_mano_guante_ch = 'Mano-Verde_cropped_df.csv' 
filename_mano_guante_df=pd.read_csv(filename_mano_guante_ch, index_col=0)
filename_mano_guante_df.head(2)

,filename,xmin,xmax,ymin,ymax,Time,Size,Type,Hand_Pixel_Width,Hand_Pixel_Height,Area,guantes,color
0,M_G_Mostrando_Mano-Verde_0_double_box1.jpg,753,909,169,294,M,G,train,156,125,19500,no,Verde
1,M_G_Mostrando_Mano-Verde_0_double_box2.jpg,1065,1216,127,269,M,G,train,151,142,21442,no,Verde


In [ ]:
#separamos las manos del archivo csv y corregimos el 'color':
color = 'mano'
#acá separamos las filas de manos de los archivos con guantes verdes/amarillos
#value_list = list_mano_amarillo_mano
value_list = list_mano_verde_mano
mano_si = filename_mano_guante_df.filename.isin(value_list)
mano_color_df = filename_mano_guante_df[mano_si].copy()
#generamos nuevas columnas, que utilizaremos mas adelante para identificar los distintos archivos:
mano_color_df['color'] = color
print(mano_color_df.shape)
mano_color_df.head(3)

(86, 13)


,filename,xmin,xmax,ymin,ymax,Time,Size,Type,Hand_Pixel_Width,Hand_Pixel_Height,Area,guantes,color
1,M_G_Mostrando_Mano-Verde_0_double_box2.jpg,1065,1216,127,269,M,G,train,151,142,21442,no,mano
2,M_G_Mostrando_Mano-Verde_11_double_box1.jpg,841,1097,390,627,M,G,train,256,237,60672,no,mano
4,M_G_Mostrando_Mano-Verde_27_double_box1.jpg,794,1011,379,606,M,G,train,217,227,49259,no,mano


In [ ]:
#guardamos la tabla en archivo csv
#name_mano_csv =  '../Mano-Amarillo_'+ color +'_cropped_df.csv'
name_mano_csv =  '../Mano-Verde_'+ color +'_cropped_df.csv'
mano_color_df.to_csv(name_mano_csv) 
print(mano_color_df.shape)

(86, 13)


In [ ]:
#separamos las manos del archivo csv y corregimos el 'guantes':
guantes = 'si'
#acá separamos las filas de manos de los archivos con guantes verdes/amarillos
#value_list = list_mano_amarillo_mano
value_list = list_mano_verde_mano
guante_si = ~filename_mano_guante_df.filename.isin(value_list)
guante_color_df = filename_mano_guante_df[guante_si].copy()
#generamos nuevas columnas, que utilizaremos mas adelante para identificar los distintos archivos:
guante_color_df['guantes'] = guantes
print(guante_color_df.shape)
guante_color_df.head(3)

(83, 13)


,filename,xmin,xmax,ymin,ymax,Time,Size,Type,Hand_Pixel_Width,Hand_Pixel_Height,Area,guantes,color
0,M_G_Mostrando_Mano-Verde_0_double_box1.jpg,753,909,169,294,M,G,train,156,125,19500,si,Verde
3,M_G_Mostrando_Mano-Verde_11_double_box2.jpg,501,736,322,573,M,G,train,235,251,58985,si,Verde
5,M_G_Mostrando_Mano-Verde_27_double_box2.jpg,577,801,330,596,M,G,train,224,266,59584,si,Verde


In [ ]:
#lo guardamos en archivo csv: 
#name_guante_csv =  '../Mano-Amarillo_amarillo_cropped_df.csv'
name_guante_csv =  '../Mano-Verde_verde_cropped_df.csv'
#guante_color_df.to_csv(name_guante_csv) 
print(guante_color_df.shape)

(83, 13)


In [ ]:
# 'Mano-Amarillo_cropped_mano_df.csv'      # shape : (87, 13)
# 'Mano-Amarillo_cropped_amarillo_df.csv'  # shape : (86, 13)
# 'Mano-Verde_cropped_mano_df.csv'         # shape : (86, 13)
# 'Mano-Verde_cropped_verde_df.csv'        # shape : (83, 13)
#  guardamos una copia del csv en sus carpetas correspondientes

In [ ]:
# nos movemos a la carpeta de clases:
os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra/classes/')
! ls Mano-*cropped_df.csv

Mano-Amarillo_amarillo_cropped_df.csv  Mano-Verde_mano_cropped_df.csv
Mano-Amarillo_mano_cropped_df.csv      Mano-Verde_verde_cropped_df.csv


In [ ]:
#volvemos a leer los archivos:
file_csv_m_g = ['Mano-Amarillo_amarillo_cropped_df.csv', 'Mano-Amarillo_mano_cropped_df.csv', 
         'Mano-Verde_mano_cropped_df.csv', 'Mano-Verde_verde_cropped_df.csv']

for file_csv in file_csv_m_g:
  print(file_csv)
  
  cropped_df = pd.read_csv(file_csv, index_col=0)
  list_filename_df = cropped_df.filename.unique().tolist()

  print('cantidad de fotos: ', len(list_filename_df)) 
  print(cropped_name_df,'shape:', cropped_df.shape, '\n')

Mano-Amarillo_amarillo_cropped_df.csv
cantidad de fotos:  86
crop/Mano-Amarillo/Mano-Amarillo_cropped_df.csv shape: (86, 13) 

Mano-Amarillo_mano_cropped_df.csv
cantidad de fotos:  87
crop/Mano-Amarillo/Mano-Amarillo_cropped_df.csv shape: (87, 13) 

Mano-Verde_mano_cropped_df.csv
cantidad de fotos:  86
crop/Mano-Amarillo/Mano-Amarillo_cropped_df.csv shape: (86, 13) 

Mano-Verde_verde_cropped_df.csv
cantidad de fotos:  83
crop/Mano-Amarillo/Mano-Amarillo_cropped_df.csv shape: (83, 13) 



Lo siguiente podríamos abordarlo juntando todas las manos y guantes. Pero para mayor control y no enloquecernos, vamos carpeta por carpeta, generando los vectores de features y luego los juntamos.

Si se animan y quieren automatizar lo siguiente, genial :)

In [ ]:
#volvemos a la carpeta classes, vemos de copiar todos los csv correspondientes a esta carpeta
os.chdir('/content/drive/My Drive/Mentoria/muestra/all_jpg_muestra/classes')
#listamos solo las carpetas
! ls -d */
! ls -d */*/

Amarillo/  Mano/  Mano-Amarillo/  Mano-Verde/  Verde/
Mano-Amarillo/Amarillo/  Mano-Verde/Mano/
Mano-Amarillo/Mano/	 Mano-Verde/Verde/


In [ ]:
#listamos los csv
extension = 'csv'
list_files_csv = [i for i in glob.glob('*_cropped_df.{}'.format(extension))]
list_files_csv = sorted(list_files_csv, reverse=False)
print(list_files_csv)

['Amarillos_cropped_df.csv', 'Mano-Amarillo_amarillo_cropped_df.csv', 'Mano-Amarillo_mano_cropped_df.csv', 'Mano-Verde_mano_cropped_df.csv', 'Mano-Verde_verde_cropped_df.csv', 'Manos_cropped_df.csv', 'Verdes_cropped_df.csv']


In [ ]:
# acá vamos cambiando los nombres de las carpetas.
#! python3 features.py Verde/

In [ ]:
# copiamos el archivo json generado
#! mv features.json features_Verdes_cropped.json

In [ ]:
#listamos los json
extension = 'json'
list_files_json = [i for i in glob.glob('*.{}'.format(extension))]
list_files_json = sorted(list_files_json, reverse=False)
list_files_json

['features_Amarillos_cropped.json',
 'features_Mano-Amarillo_amarillo_cropped.json',
 'features_Mano-Amarillo_mano_cropped.json',
 'features_Mano-Verde_mano_cropped.json',
 'features_Mano-Verde_verde_cropped.json',
 'features_Manos_cropped.json',
 'features_Verdes_cropped.json']

In [ ]:
#Leemos y ordenamos por nombre los archivos csv:
for csv_df in list_files_csv:
  cropped_df = pd.read_csv(csv_df, index_col=0).sort_values(by=['filename'])
  print(csv_df, ':', cropped_df.shape)
  

Amarillos_cropped_df.csv : (428, 13)
Mano-Amarillo_amarillo_cropped_df.csv : (86, 13)
Mano-Amarillo_mano_cropped_df.csv : (87, 13)
Mano-Verde_mano_cropped_df.csv : (86, 13)
Mano-Verde_verde_cropped_df.csv : (83, 13)
Manos_cropped_df.csv : (489, 13)
Verdes_cropped_df.csv : (409, 13)


In [ ]:
#acá armamos todos los archivos con los features de cada una de las carpetas:
for json_file in list_files_json:
  json_csv_name = json_file[:-5]+'_json_df.csv'
  print(json_csv_name)

  df = pd.read_json(json_file)
  df_json_transpose = df.T.reset_index().copy()
  #print(df_json_transpose.shape)

  #para listar solo los nombres de los '.jpg'
  list_index_json = df_json_transpose['index']
  #print(len(list_index_json))

  index_json = []

  for file_jpg in list_index_json:
    x = re.search('[A-Z]\w+_.+jpg', file_jpg)
    index_json.append(x.group())

  #print(len(index_json)) 

  df_json_transpose['filename'] = index_json
  df_json_transpose = df_json_transpose.drop(['index'], axis=1)
  df_json_transpose = df_json_transpose.set_index('filename')
  df_json_transpose = df_json_transpose.reset_index().sort_values(by=['filename']).copy()
  print(df_json_transpose.shape)
  #df_json_transpose.to_csv(json_csv_name)

features_Amarillos_cropped_json_df.csv
(428, 2049)
features_Mano-Amarillo_amarillo_cropped_json_df.csv
(86, 2049)
features_Mano-Amarillo_mano_cropped_json_df.csv
(87, 2049)
features_Mano-Verde_mano_cropped_json_df.csv
(86, 2049)
features_Mano-Verde_verde_cropped_json_df.csv
(83, 2049)
features_Manos_cropped_json_df.csv
(489, 2049)
features_Verdes_cropped_json_df.csv
(409, 2049)


In [ ]:
#listamos los json
extension = 'csv'
list_files_json_csv = [i for i in glob.glob('*cropped_json_df.{}'.format(extension))]
list_files_json_csv = sorted(list_files_json_csv, reverse=False)
list_files_json_csv

['features_Amarillos_cropped_json_df.csv',
 'features_Mano-Amarillo_amarillo_cropped_json_df.csv',
 'features_Mano-Amarillo_mano_cropped_json_df.csv',
 'features_Mano-Verde_mano_cropped_json_df.csv',
 'features_Mano-Verde_verde_cropped_json_df.csv',
 'features_Manos_cropped_json_df.csv',
 'features_Verdes_cropped_json_df.csv']

In [ ]:
result_name = csv_df[:-7] + '_features_labels_df.csv'
result_name

'Verdes_cropped_features_labels_df.csv'

In [ ]:
for csv_df, json_csv_file in zip(list_files_csv, list_files_json_csv):
  left  = pd.read_csv(csv_df, index_col=0).sort_values(by=['filename'])
  right = pd.read_csv(json_csv_file, index_col=0).sort_values(by=['filename'])
  print('left', csv_df, ':', left.shape)
  print('right', json_csv_file, ':', right.shape)
  result = pd.merge(left, right, on='filename')
  result_name_df = result.copy()
  print(result_name_df.shape)
  result_name = csv_df[:-7] + '_features_labels_df.csv'
  #result_name_df.to_csv(result_name) 
  

left Amarillos_cropped_df.csv : (428, 13)
right features_Amarillos_cropped_json_df.csv : (428, 2049)
(428, 2061)
left Mano-Amarillo_amarillo_cropped_df.csv : (86, 13)
right features_Mano-Amarillo_amarillo_cropped_json_df.csv : (86, 2049)
(86, 2061)
left Mano-Amarillo_mano_cropped_df.csv : (87, 13)
right features_Mano-Amarillo_mano_cropped_json_df.csv : (87, 2049)
(87, 2061)
left Mano-Verde_mano_cropped_df.csv : (86, 13)
right features_Mano-Verde_mano_cropped_json_df.csv : (86, 2049)
(86, 2061)
left Mano-Verde_verde_cropped_df.csv : (83, 13)
right features_Mano-Verde_verde_cropped_json_df.csv : (83, 2049)
(83, 2061)
left Manos_cropped_df.csv : (489, 13)
right features_Manos_cropped_json_df.csv : (489, 2049)
(489, 2061)
left Verdes_cropped_df.csv : (409, 13)
right features_Verdes_cropped_json_df.csv : (409, 2049)
(409, 2061)


In [ ]:
#listamos los csv
extension = 'csv'
list_files_features_csv = [i for i in glob.glob('*features_labels_df.{}'.format(extension))]
list_files_features_csv = sorted(list_files_features_csv, reverse=False)
print(list_files_features_csv)
print(len(list_files_features_csv))

['Amarillos_cropped_features_labels_df.csv', 'Mano-Amarillo_amarillo_cropped_features_labels_df.csv', 'Mano-Amarillo_mano_cropped_features_labels_df.csv', 'Mano-Verde_mano_cropped_features_labels_df.csv', 'Mano-Verde_verde_cropped_features_labels_df.csv', 'Manos_cropped_features_labels_df.csv', 'Verdes_cropped_features_labels_df.csv']
7


In [ ]:
#juntamos todos los arhivos en una hermosa y gigante tabla final!! 
csv_wlabels = (pd.read_csv(f, index_col=0) for f in list_files_features_csv)
csv_wlabels_muestra_df = pd.concat(csv_wlabels, ignore_index=True)
print(csv_wlabels_muestra_df.shape)
csv_wlabels_muestra_df.head()
#csv_wlabels_muestra_df.to_csv('csv_wlabels_muestra_df.csv') 

In [ ]:
#leemos el archivo de la tabla:
csv_wlabels_df = pd.read_csv('csv_wlabels_muestra_df.csv', index_col=0)
print(csv_wlabels_df.shape)
csv_wlabels_df.head()

(1668, 2061)


,filename,xmin,xmax,ymin,ymax,Time,Size,Type,Hand_Pixel_Width,Hand_Pixel_Height,Area,guantes,color,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,M_G_Mostrando_Amarillos_0_double_box1.jpg,218,461,464,715,M,G,train,243,251,60993,si,Amarillo,0.740974,1.066019,0.132695,0.224892,0.140456,0.129134,0.201447,1.076733,0.461108,0.299800,0.457940,0.201904,0.569677,0.737381,0.480929,0.200076,0.018410,0.674050,0.028826,0.005310,0.128693,0.592353,0.138541,0.141684,0.363294,0.264674,0.519292,...,0.463567,0.091676,0.300955,0.696112,2.409413,0.536980,0.102273,0.157441,0.165093,1.067619,0.751934,0.231443,0.487437,0.446320,0.822407,0.527523,0.217110,0.489194,1.535306,0.521390,0.599480,0.201182,0.391337,0.040206,1.347440,0.448389,0.462778,0.590751,0.877142,0.009986,0.939221,0.859877,0.981910,0.356366,1.748501,0.288037,0.079172,0.266234,0.078312,0.421612
1,M_G_Mostrando_Amarillos_0_double_box2.jpg,617,865,432,598,M,G,train,248,166,41168,si,Amarillo,1.621832,0.208859,0.627041,0.061717,0.052486,0.064176,0.479256,1.068516,0.509059,0.207385,0.218195,0.169190,0.695486,0.237641,0.146711,0.126334,0.029900,0.148363,0.126833,0.181351,0.044585,0.447270,0.134360,0.113650,1.153900,0.653832,0.041024,...,0.816892,0.027489,0.183116,0.626535,1.734605,0.672962,0.030057,0.101685,0.295544,0.392251,0.874054,0.105285,0.110713,0.421417,1.335695,0.500971,1.139208,0.110803,0.850265,0.618181,0.077352,0.376629,0.319234,0.783792,0.427963,0.133063,0.064981,0.017857,0.320706,0.219192,1.199712,0.808466,0.491182,0.356616,0.812572,0.481357,0.071471,0.572322,0.052072,0.213453
2,M_G_Mostrando_Amarillos_105_double_box1.jpg,875,1169,321,450,M,G,train,294,129,37926,si,Amarillo,0.777015,0.350298,0.523675,0.141681,0.008227,0.364888,0.559975,1.060057,0.648008,0.075809,0.419526,0.200864,0.553840,0.182195,0.244877,0.038389,0.046786,0.020999,0.281995,0.016088,0.135179,0.635030,0.276898,0.014077,0.613297,0.622520,0.117482,...,0.250315,0.025067,0.150927,0.604435,1.067641,0.847809,0.220889,0.298962,0.535717,0.566534,0.531686,0.208911,0.254966,0.229268,1.809620,0.362365,0.468568,0.055350,1.388036,0.884211,0.340101,0.305252,0.343156,1.061346,1.043128,0.004126,0.033861,0.020319,0.621639,0.040598,0.916142,0.246840,0.688755,0.101427,0.917090,0.407893,0.343683,0.428996,0.080831,0.005678
3,M_G_Mostrando_Amarillos_105_double_box2.jpg,502,758,145,307,M,G,train,256,162,41472,si,Amarillo,0.206803,0.032262,0.783582,0.104788,0.198923,0.229943,0.102975,1.872409,0.646370,0.369061,0.323460,0.136720,0.515654,0.139382,0.084579,0.205710,0.007865,0.329643,0.279838,0.096330,0.003207,0.161915,0.546153,0.034820,0.178539,0.245108,0.190008,...,0.146642,0.114052,0.005513,0.370992,0.665213,1.402337,0.162487,0.155748,0.241372,0.702011,0.993891,0.174953,0.551115,0.294757,0.660277,0.545881,0.343379,0.348534,0.342936,0.632171,0.079185,0.238604,0.062875,0.798502,1.009929,0.130553,0.003187,0.115578,0.427998,0.110039,0.992854,0.904692,0.112806,0.488674,1.150740,0.275088,0.254404,0.455159,0.012721,0.091329
4,M_G_Mostrando_Amarillos_114_double_box1.jpg,916,1151,285,463,M,G,train,235,178,41830,si,Amarillo,0.683246,0.319097,0.322898,0.489538,0.152056,0.593756,0.177822,1.093265,0.949665,0.154196,0.645907,0.270926,0.383814,0.204949,0.398145,0.102344,0.060811,0.121956,0.028302,0.120451,0.244606,0.387735,0.251303,0.236359,0.880333,0.466272,0.185311,...,0.652505,0.140662,0.354066,0.571745,2.067905,1.038591,0.099552,0.280820,0.516782,0.820352,0.601326,0.042655,0.119472,0.094523,2.042640,0.790532,0.266744,0.056142,0.620942,0.859705,0.439731,0.290626,0.310608,0.721135,0.988298,0.026431,0.317181,0.199194,0.524838,0.139992,1.304742,0.415517,0.652986,0.141079,1.024287,0.117491,0.007151,0.294575,0.217550,0.240535


Para analizar esta tabla (con algunas modificaciones, acorde a la materia *'Análisis y Curación'*) pueden ir a la notebook: *[3-Análisis_y_Curación-Detección_de_Objetos](https://colab.research.google.com/drive/1E6ZUVrYo_n-YS5Hfc5MdIvB9j03ayJXi?usp=sharing)'*